<a href="https://colab.research.google.com/github/SanskarLoganDev/civic-hack-taskmasters/blob/main/civic_hack_keyword_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q vaderSentiment
!pip install -q fuzzywuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 574.9 kB/s eta 0:00:00


In [2]:
# Mount google drive directory
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
# Standard library imports
import re
import pickle
from datetime import datetime

# Related third party imports
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import spacy
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from fuzzywuzzy import fuzz

# nltk specific downloads
nltk.data.path.append('/content/')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# nltk specific imports
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

# Local application/library specific imports
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import AutoTokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
PATH = 'drive/MyDrive/Civic Hack/'
DATASET_PATH = PATH + 'DEA-2024-0059.csv'

In [ ]:
# Load dataset
df = pd.read_csv(DATASET_PATH)

In [11]:
df.head()

id  \
0  DEA-2024-0059-14349   
1  DEA-2024-0059-42529   
2  DEA-2024-0059-16374   
3   DEA-2024-0059-6467   
4  DEA-2024-0059-40514   

                                                       api_url  \
0  https://api.regulations.gov/v4/comments/DEA-2024-0059-14349   
1  https://api.regulations.gov/v4/comments/DEA-2024-0059-42529   
2  https://api.regulations.gov/v4/comments/DEA-2024-0059-16374   
3   https://api.regulations.gov/v4/comments/DEA-2024-0059-6467   
4  https://api.regulations.gov/v4/comments/DEA-2024-0059-40514   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42813 entries, 0 to 42812
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               42813 non-null  object
 1   api_url          42813 non-null  object
 2   comment          42807 non-null  object
 3   firstName        42259 non-null  object
 4   lastName         42256 non-null  object
 5   modifyDate       42813 non-null  object
 6   objectId         42813 non-null  object
 7   organization     448 non-null    object
 8   postedDate       42813 non-null  object
 9   reasonWithdrawn  6 non-null      object
 10  receiveDate      42807 non-null  object
 11  title            42813 non-null  object
 12  trackingNbr      42807 non-null  object
 13  withdrawn        42813 non-null  bool  
dtypes: bool(1), object(13)
memory usage: 4.3+ MB


In [13]:
df.describe()

id  \
count                 42813   
unique                42813   
top     DEA-2024-0059-23691   
freq                      1   

                                                            api_url  \
count                                                         42813   
unique                                                        42813   
top     https://api.regulations.gov/v4/comments/DEA-2024-0059-23691   
freq                                                              1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [14]:
df.isna().sum()

,0
id,0
api_url,0
comment,6
firstName,554
lastName,557
modifyDate,0
objectId,0
organization,42365
postedDate,0
reasonWithdrawn,42807


In [15]:
df.duplicated().sum()

np.int64(0)

In [17]:
df.drop('firstName', axis=1, inplace=True)
df.drop('lastName', axis=1, inplace=True)
df.drop('organization', axis=1, inplace=True)
df.drop('reasonWithdrawn', axis=1, inplace=True)

In [18]:
df['comment'] = df['comment'].fillna("")

In [19]:
df.isna().sum()

,0
id,0
api_url,0
comment,0
modifyDate,0
objectId,0
postedDate,0
receiveDate,6
title,0
trackingNbr,6
withdrawn,0


## Data Cleaning

In [27]:
import html

# Load spaCy's small English model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Step 1: Convert to string if not already
    if not isinstance(text, str):
        text = str(text)

    # Step 2: Remove markdown and URLs
    text = re.sub(r'\!\[.*?\]\(.*?\)|https?://\S+|www\.\S+', '', text)

    # Step 3: Remove extra line breaks and normalize whitespace
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Step 4: Remove HTML encodings like '&gt;' and '&amp;'
    text = re.sub(r"&\w+;", "", text)

    # Step 5: Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Step 6: Convert HTML-encoded characters (e.g., &#39; → ')
    text = html.unescape(text)

    # Step 7: Remove trailing numeric scores or tabs (if needed)
    text = re.sub(r'\t\d+\.\d+$', '', text)

    return text

df['clean_text'] = df['comment'].apply(preprocess_text)

In [28]:
df.head()

id  \
0  DEA-2024-0059-14349   
1  DEA-2024-0059-42529   
2  DEA-2024-0059-16374   
3   DEA-2024-0059-6467   
4  DEA-2024-0059-40514   

                                                       api_url  \
0  https://api.regulations.gov/v4/comments/DEA-2024-0059-14349   
1  https://api.regulations.gov/v4/comments/DEA-2024-0059-42529   
2  https://api.regulations.gov/v4/comments/DEA-2024-0059-16374   
3   https://api.regulations.gov/v4/comments/DEA-2024-0059-6467   
4  https://api.regulations.gov/v4/comments/DEA-2024-0059-40514   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [29]:
from textblob import TextBlob

# Convert 'clean_text' column to string type before applying TextBlob
df['clean_text'] = df['clean_text'].astype(str)

# Calculate sentiment polarity
df['textblob_sentiment_score'] = df['clean_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [30]:
df[['clean_text', 'textblob_sentiment_score']]

clean_text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [31]:
nltk.download('stopwords')
nltk.download('wordnet')

def tokenize_text(text):
    if not isinstance(text, str):
        return ""

    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()  # Convert to lowercase

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return " ".join(words)

df['processed_text'] = df['clean_text'].apply(tokenize_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
df['processed_text']

,processed_text
0,legalize marijuana support reschedulingits proven people get heavily addicted opiate painkiller prescribed doctor several report individual moving harder drug heroin doctor cut prescription noted people kill people drug marijuana one go life threatening withdrawal arent able get marijuana one go kill someone get hit marijuana reschedule marijuana work toward legalizing use tax revenue toward supporting community properly educating child true effect drug safe
1,thank opportunity comment dea proposed rule status marijuana controlled substance act csa appreciate biden administration effort reconsider marijuana inappropriate classification schedule controlled substance deep concern insufficiency rescheduling instead keeping marijuana illegal csa time end country failed policy criminalization completely removing marijuana csas drug schedule urge dea reconsider proposed rule issue new rule deschedule marijuana csarescheduling marijuana schedule iii enough marijuana must removed csa federally regulated medical adult use rescheduling end federal marijuana criminalization rescheduling end federal marijuana arrest even possession use rescheduling release anyone prison marijuana rescheduling expunge previous marijuana arrest rescheduling end deportation immigration consequencesor tourist visa restriction stemming marijuana activity rescheduling restore access government benefit people lost due marijuana activity rescheduling bring state marijuana program compliance federal law rescheduling guarantee fair working condition individual working marijuana industry rescheduling facilitate patient access medical marijuana overall schedule iii insufficient even marijuana industry example although placing marijuana schedule iii would remove unfair onerous federal tax penalty upon stateregulated marijuana business wont give legal access essential business service unlikely result significant wage increase worker lower price patient consumer importantly rescheduling marijuana would continue criminalization marijuana activity occurring united state personal possession cultivation use participation statelegal marijuana business programsthe way end federal marijuana criminalization harm completely remove marijuana csa descheduling guided principle public health equity new federal marijuana policy seek end unjust criminal legal consequence marijuana activity restore right opportunity individual community targeted marijuana criminalization ensure diverse marketplace protecting small business equity program emergence national multinational corporate monopoly outcome schedule iii insufficient process evaluating marijuana status csa also flawed administrative review process conducted fda dea design limited scope fails consider racist origin impact marijuana prohibition produced overwhelming negative social economic public health consequence generational outcome still felt predominantly black latino community experiencing mass incarceration inaccessible public housing denial food assistance program among detrimental consequence continue marijuana rescheduling moreover decade u played instrumental role creating enforcing marijuana prohibition global level devastating consequence around world changing reality domestic level come responsibility start repairing damage caused prohibitionist policy support initiative lead systemic change international arena consequently u undertake active diplomacy united nation level together likeminded country challenge outdated global drug control regimegiven biden administration repeated acknowledgment racially discriminatory enforcement marijuana law related harm process biden administration develop new federal policy marijuana without consideration unjustifiablepresident biden repeatedly promised decriminalize marijuana call dea undermine president bidens campaign pledge finalizing proposed rule reschedule marijuana csa imperative biden administration follow unfulfilled commitment taking action end repair h

# Vectorization

In [33]:
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')

In [35]:
def print_topics(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print("\n")

def calculate_topic_diversity(lda_model, feature_names, top_n=10):
    topic_words = []
    for topic in lda_model.components_:
        top_words = [feature_names[i] for i in topic.argsort()[:-top_n - 1:-1]]
        topic_words.append(set(top_words))

    n_topics = len(topic_words)
    total_pairs = n_topics * (n_topics - 1) / 2
    diversity = 0
    for i in range(n_topics):
        for j in range(i + 1, n_topics):
            diversity += len(topic_words[i].intersection(topic_words[j])) / top_n
    return 1 - (diversity / total_pairs)

def create_lda_model(n_topics, vectorizer, processed_text):

    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)

    keywords = vectorizer.fit_transform(processed_text)
    lda.fit(keywords)

    # Get the words corresponding to the topics
    feature_names = vectorizer.get_feature_names_out()

    n_top_words = 10  # Number of top words per topic
    print_topics(lda, feature_names, n_top_words)

    topic_distributions = lda.transform(keywords)  # (n_docs, n_topics)

    return lda, keywords, topic_distributions, vectorizer

def print_metrics(lda_model, keywords):
    perplexity = lda_model.perplexity(keywords)
    print(f"Perplexity: {perplexity}")

    log_likelihood = lda_model.score(keywords)
    print(f"Log-Likelihood: {log_likelihood}")

def get_silohouette_score(lda_model, topic_distributions, keywords):
    topic_distributions = lda_model.transform(keywords)  # (n_docs, n_topics)
    silhouette = silhouette_score(topic_distributions, np.argmax(topic_distributions, axis=1))
    print(f"Silhouette Score: {silhouette:.2f}")

In [36]:
tfidf_lda, tfidf_keywords, tfidf_topic_distributions, tfidf = create_lda_model(10, tfidf, df['processed_text'])

Topic #1:
marijuana use hhs botanical cannabis user disorder likely high vomiting


Topic #2:
police impairment marijuana driving officer safety impaired regulation lack risk


Topic #3:
cannabis medical hhs schedule safety ii substance botanical association marijuana


Topic #4:
marijuana use marketing advertising legalize tobacco disorder likely hospital teen


Topic #5:
cannabis people criminalization human step housing descheduling lead enforcement fundamental


Topic #6:
cannabis alcohol medical thousand american lower year far academy trial


Topic #7:
marijuana reschedule schedule fully cannabis deschedule rescheduling support drug completely


Topic #8:
cannabis marijuana alcohol help life plant like schedule drug people


Topic #9:
marijuana reclassifying scientific substance proposal cannabis schedule step drug potential


Topic #10:
marijuana rescheduling end rule biden csa criminalization federal activity administration




In [37]:
print_metrics(tfidf_lda, tfidf_keywords)

Perplexity: 309.3543920788682
Log-Likelihood: -2015067.009595354


In [39]:
# Add dominant topic to the DataFrame
df['tf5_dominant_topic'] = tfidf_topic_distributions.argmax(axis=1)
df[['comment', 'tf5_dominant_topic']]

comment  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [45]:
df['processed_text'] = df['processed_text'].astype(str)

In [ ]:
from transformers import pipeline

# Load a T5 summarization pipeline
summarizer = pipeline("summarization", model="t5-small")

# Give T5 a "story" or a sentence (works best with short passages, but can use keywords as a hack)
# text = f"Summarize: {keywords}" # This line is not needed

# Generate summary
summary = summarizer(df['processed_text'].tolist(), max_new_tokens=30, max_length=10, min_length=5, do_sample=False)
print("Summary:", summary[0]['summary_text'])

In [58]:
from openai import OpenAI
from openai.types.chat import ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam

client = OpenAI(
    base_url = "http://Bedroc-Proxy-F5ZWJCQVWCrD-424352376.us-east-1.elb.amazonaws.com/api/v1",
    api_key = "bedrock-api-key-YmVkcm9jay5hbWF6b25hd3MuY29tLz9BY3Rpb249Q2FsbFdpdGhCZWFyZXJUb2tlbiZYLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFTSUFUNUJaWVNWSUpZM1kyTlJFJTJGMjAyNTA3MjYlMkZ1cy1lYXN0LTElMkZiZWRyb2NrJTJGYXdzNF9yZXF1ZXN0JlgtQW16LURhdGU9MjAyNTA3MjZUMTYzMzU4WiZYLUFtei1FeHBpcmVzPTQzMjAwJlgtQW16LVNlY3VyaXR5LVRva2VuPUlRb0piM0pwWjJsdVgyVmpFRGthQ1hWekxXVmhjM1F0TVNKSU1FWUNJUUNERkw3WW9KZHFCRFVNb1h5NkpKbUZkek00Q2g4OU1mMFJOVlhQTEJ2MUl3SWhBSmcyNWtNR3dFa3VWbFBJQVFyRGlybXJWUCUyQlN2bjMxOTB4OGRCdiUyQnJkQXJLcVVEQ0dJUUFCb01Nalk0TlRVMk5qQTBOelV5SWd4RmVFciUyQjhWckwzUE9ZOHEwcWdnT2w4bjhveEY1MFFxbFJTWU1lMDBkc3VNcUtLWSUyRkJJTGlYUCUyQlRzYTV4ejJjSnNzRnRGUEZUUlZCemh1NmNzYTE4bURNNUZYeGpSUmJKOEZkS2lJa1RuMVNheU5HYW4zVHZESHB4RU1mbGw1RXA1aXRkVnlZYWxQSjElMkJLWVlLT21HdiUyQnU3YnNIN3ZhYVolMkI3ZVQwWTVDMXhGNVU2ejM4bFduelk2ZTdrQVVDSmtJQWVUclFJenJNV3NiYUVxV1FvYjhEdnVVaGRabnZTSXE3dzljSnRMcGVVUnpaUjl3JTJGNWRCJTJCMUpYN3hSVURwR050Q1VTb0lUTDdCZUlVdUswSXVWcjVQcVQ5YjZuenNLdWplTSUyRkNKUVFXYTl5UnJqUUs1MW5WYnMxS1A1MzhFZ0hGQlZaRFRWJTJGTHNITTVUZDVRalBuTnpMZmgxdGtsWHk5TGJqMUJUbm90eEU3SnY4VVNTQ3p4UnptV0EwdlB5JTJCdXVUeld0RDVZWld1bWFmTlF2aGJOaGlBZ24lMkJKU0RUVElmRjA5U1MzdllxY1ZPRVJsQWtUVEtaT2xhVGxsNnNVS3pjRDBIRmVTV0FsOUpVaUxBY2JlT0hHblVNblc2UnpBc3IzVkcwNlppUHNPVFRDQXVKJTJGeXUwRndVVE9kUjlkclM5d3NsRFlVeWt4MGFGRWclMkJibHMzTkRDVmdaVEVCanJkQXJVMkJmZDdNRHF0YVFubzRhVllkU1k2QTMxdzN4YVZqQUhPZzgxdGVwdWFPZGt0T1FTcFVkQXQzSTclMkY4Q2pNcVFZb3dzNXNCS3NCaDNzVFJCZGxMdkY3dkZrSFYwRTQwVzZYeHpXWDJqTjhla1VCOTBwMkZtcHNBclZBWER4b2Y1JTJGbDdsRU9DMDMxTk9UdG9jbHQ3blJqV1ZwQTRCSVNrM2tWeFJyNTVGWndCS2lPRnllUmJtOVJPWEQ5dU5ZUmtFZnNwJTJGOFZRcVpSVCUyQlZsWXgySjgxeGpYeGhCUG15SDVhTHVPdzZOeGhtQ3FqV0dvMEVwZ1dqS1ZMamIlMkJsM01rb0FjWFVvJTJGTERrUXVpejdSakpWRHJYSDclMkZKNktob1ZCMEtNc0VISEx0Qk5tYWhpT1VWWEc1NnY1YVBFVEhqOWNaZThOSnBmOVB1cndBdGRuVEM5VWglMkZBUjlqaUp1OXZvWG1JSVIxU1JpMDF4em1QUTNjemQ4cHFZUVVNQ1NVSTlROEp1RzNDZnMzVTRzdUt2Wk5wWW5KZVc2NUFOcElHQVAxV1ViVndrTEZONXFNa0JiNzlqaVJGd2drN3NXVmtvckglMkI2dkpsb2J5ZWFBUUVHcVUlM0QmWC1BbXotU2lnbmF0dXJlPTQ0ZGRhNWI1ZTllNjUxODI5YjJlYzQ5NjA4YTIwOWY2YTQ2ZTgwN2QzNzk4YjU3OGI1ZjMzY2NmNDBjZmY4MTcmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0JlZlcnNpb249MQ==",
)

In [ ]:
import time

def summarize_text(text):
  try:
    system_message = ChatCompletionSystemMessageParam(
      role="system",
      content=(
          "You are a public policy expert that writes concise summaries. "
      ),
    )

    user_message = ChatCompletionUserMessageParam(
      role="user",
      content=f"Write a summary about \"{text}\". "
              "Make it informative, clear, and suitable for a general audience."
    )

    time.sleep(1)

    response = client.chat.completions.create(
      model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
      messages=[system_message, user_message],
      temperature=0.8,
      max_tokens=600,
    )

    content = response.choices[0].message.content

    return content
  except Exception as e:
    print(f"Error: {e}")
    return ""

df['summary'] = df['processed_text'].apply(summarize_text)

Error: Error code: 429 - {'detail': 'An error occurred (ThrottlingException) when calling the Converse operation: Too many requests, please wait before trying again.'}
